In [26]:
import pandas as pd
import json
import sys

# Cargar los archivos JSON
json1_path = 'D:/DataScience/TFB/Data/all_anime_data1.json'
json2_path = 'D:/DataScience/TFB/Data/all_anime_data2.json'

# Leer el primer archivo JSON
with open(json1_path, 'r', encoding='utf-8') as f:
    data1 = json.load(f)

# Leer el segundo archivo JSON
with open(json2_path, 'r', encoding='utf-8') as f:
    data2 = json.load(f)

# Combinar los datos (asumiendo que son listas de diccionarios)
combined_data = data1 + data2

# Crear el DataFrame
df = pd.DataFrame(combined_data)

# Ahora puedes proceder con la limpieza de datos
pd.set_option('display.max_columns', None)
print(df.head())
print(f"Total de registros combinados: {len(df)}")

                                                data  status type message  \
0  {'mal_id': 1, 'url': 'https://myanimelist.net/...     NaN  NaN     NaN   
1  {'mal_id': 5, 'url': 'https://myanimelist.net/...     NaN  NaN     NaN   
2  {'mal_id': 6, 'url': 'https://myanimelist.net/...     NaN  NaN     NaN   
3  {'mal_id': 7, 'url': 'https://myanimelist.net/...     NaN  NaN     NaN   
4  {'mal_id': 8, 'url': 'https://myanimelist.net/...     NaN  NaN     NaN   

  error  
0   NaN  
1   NaN  
2   NaN  
3   NaN  
4   NaN  
Total de registros combinados: 27530


In [27]:
df = pd.json_normalize(df['data'])

In [28]:
print(df.head())

   mal_id                                                url approved  \
0     1.0       https://myanimelist.net/anime/1/Cowboy_Bebop     True   
1     5.0  https://myanimelist.net/anime/5/Cowboy_Bebop__...     True   
2     6.0             https://myanimelist.net/anime/6/Trigun     True   
3     7.0  https://myanimelist.net/anime/7/Witch_Hunter_R...     True   
4     8.0     https://myanimelist.net/anime/8/Bouken_Ou_Beet     True   

                                              titles  \
0  [{'type': 'Default', 'title': 'Cowboy Bebop'},...   
1  [{'type': 'Default', 'title': 'Cowboy Bebop: T...   
2  [{'type': 'Default', 'title': 'Trigun'}, {'typ...   
3  [{'type': 'Default', 'title': 'Witch Hunter Ro...   
4  [{'type': 'Default', 'title': 'Bouken Ou Beet'...   

                             title            title_english  \
0                     Cowboy Bebop             Cowboy Bebop   
1  Cowboy Bebop: Tengoku no Tobira  Cowboy Bebop: The Movie   
2                           Trigun 

1. Normalización de Columnas Anidadas

In [29]:
# Función para extraer valores de listas de diccionarios
def extract_from_list_dict(data, key='name'):
    if isinstance(data, list):
        return ', '.join([str(d.get(key, '')) for d in data])
    return ''

# Aplicar a columnas relevantes
for col in ['titles', 'producers', 'studios', 'genres', 'themes', 'demographics', 'licensors']:
    df[f'{col}_extracted'] = df[col].apply(extract_from_list_dict)

# Extraer imagen principal
df['main_image'] = df['images.jpg.large_image_url']

2. Manejo de Datos Temporales

In [30]:
# Convertir fechas a datetime
df['aired_from'] = pd.to_datetime(df['aired.from'])
df['aired_to'] = pd.to_datetime(df['aired.to'])

# Crear columna de década
df['decade'] = (df['aired.prop.from.year']//10)*10

3. Limpieza de Columnas Redundantes

In [31]:
# Versión segura que maneja tipos y valores nulos
df['title_consistency'] = df.apply(
    lambda x: (
        str(x['title']) in str(x['titles_extracted']).split(', ') 
        if pd.notna(x['title']) and pd.notna(x['titles_extracted']) 
        else False
    ), 
    axis=1
)

# Alternativa más eficiente (evitando apply)
df['title_consistency'] = [
    str(title) in titles.split(', ') 
    for title, titles in zip(df['title'], df['titles_extracted'])
]

4. Tratamiento de Valores Nulos

In [32]:
# Imputar valores faltantes estratégicamente
df['score'] = df['score'].fillna(df.groupby('type')['score'].transform('median'))
df['duration'] = df['duration'].fillna('Unknown')
df['type'] = df['type'].fillna('Unknown')

# Eliminar filas con datos críticos faltantes
df = df.dropna(subset=['mal_id', 'title'])

In [33]:
import pandas as pd
from datetime import datetime

# ================= CONFIGURACIÓN INICIAL =================
TIME_SLOTS = {
    1: ('morning', (5, 12)),
    2: ('midday', (12, 14)),
    3: ('afternoon', (14, 19)),
    4: ('prime', (19, 23)),
    5: ('night', (23, 24)),
    6: ('late_night', (0, 5)),
    0: ('unknown', (None, None))
}

# ================= FUNCIONES AUXILIARES =================
def get_season_from_date(date):
    try:
        month = pd.to_datetime(date).month
        if 3 <= month <= 5: return 'Spring'
        elif 6 <= month <= 8: return 'Summer'
        elif 9 <= month <= 11: return 'Fall'
        else: return 'Winter'
    except:
        return None

def get_time_slot(time_str):
    if pd.isna(time_str): return 'unknown'
    try:
        hour = int(pd.to_datetime(time_str).hour)
        for slot_id, (name, (start, end)) in TIME_SLOTS.items():
            if slot_id == 0: continue
            if (start <= hour < end) or (name == 'late_night' and (hour >= 0 or hour < 5)):
                return name
        return 'unknown'
    except:
        return 'unknown'

def extract_year_from_broadcast(row):
    try:
        if pd.notna(row['aired_from']):
            return pd.to_datetime(row['aired_from']).year
        if pd.notna(row.get('broadcast.string')):
            years = [int(s) for s in str(row['broadcast.string']).split() if s.isdigit()]
            if years: return max(years)
        if pd.notna(row.get('aired.prop.from.year')):
            return int(row['aired.prop.from.year'])
    except:
        pass
    return None

# ================= PROCESAMIENTO PRINCIPAL =================
# 1. Crear copia segura
df_clean = df.copy()

# 2. Calcular columnas nuevas
df_clean['season'] = df_clean['aired_from'].apply(get_season_from_date)
df_clean['time_slot'] = df_clean['broadcast.time'].apply(get_time_slot)
df_clean['year_computed'] = df_clean.apply(extract_year_from_broadcast, axis=1)

# 3. Consolidar columnas
df_clean['season'] = df_clean['season'].astype('category')
df_clean['time_slot'] = pd.Categorical(
    df_clean['time_slot'],
    categories=['morning', 'midday', 'afternoon', 'prime', 'night', 'late_night', 'unknown'],
    ordered=True
)
df_clean['year'] = df_clean['year'].fillna(df_clean['year_computed']).astype('Int32')

# 4. Eliminar columnas temporales/intermedias
cols_to_drop = [
    'season_computed', 'year_computed', 
    'aired.from', 'aired.to', 'aired.prop.from.day', 'aired.prop.from.month',
    'aired.prop.from.year', 'aired.prop.to.day', 'aired.prop.to.month',
    'aired.prop.to.year', 'aired.string', 'broadcast.day', 'broadcast.time',
    'broadcast.timezone', 'broadcast.string',
    # Columnas de imágenes
    'images.jpg.image_url', 'images.jpg.small_image_url', 'images.jpg.large_image_url',
    'images.webp.image_url', 'images.webp.small_image_url', 'images.webp.large_image_url',
    
    # Columnas de títulos redundantes (conservando solo title y titles_extracted)
    'title_english', 'title_japanese', 'titles', 'title_english', 'title_japanese', 'title_synonym',
    # Columnas de trailer
    'trailer.youtube_id', 'trailer.url', 'trailer.embed_url',
    'trailer.images.image_url', 'trailer.images.small_image_url',
    'trailer.images.medium_image_url', 'trailer.images.large_image_url',
    'trailer.images.maximum_image_url',
    
    # Columnas anidadas originales (usaremos las versiones _extracted)
    'genres', 'themes', 'explicit_genres', 'demographics',
    'producers', 'studios', 'licensors'
]
df_clean = df_clean.drop(columns=[col for col in cols_to_drop if col in df_clean.columns])

# 5. Optimización de tipos
dtype_map = {
    'mal_id': 'uint32',
    'episodes': 'uint16',
    'score': 'float32',
    'rank': 'uint32',
    'popularity': 'uint32',
    'members': 'uint32',
    'favorites': 'uint32',
    'decade': 'uint16'
}
df_clean = df_clean.astype(dtype_map, errors='ignore')

# 6. Reordenar columnas
first_cols = [
    'mal_id', 'title', 'type', 'year',
    'season', 'time_slot', 'decade', 'score', 'rank', 'popularity', 'members', 'favorites'
]
other_cols = [col for col in df_clean.columns if col not in first_cols]
df_clean = df_clean[first_cols + other_cols]


# ================= NORMALIZACIÓN DE GÉNEROS =================
print("\n🔨 Procesando géneros y temas...")

# 1. Extraer TODOS los géneros únicos
all_genres = df_clean['genres_extracted'].str.split(', ').explode().dropna().unique()
genres_df = pd.DataFrame({
    'genre_id': range(1, len(all_genres)+1),
    'genre_name': sorted(all_genres)
})

# 2. Extraer TODOS los temas únicos
all_themes = df_clean['themes_extracted'].str.split(', ').explode().dropna().unique()
themes_df = pd.DataFrame({
    'theme_id': range(1, len(all_themes)+1),
    'theme_name': sorted(all_themes)
})

all_licensors = df_clean['licensors_extracted'].str.split(', ').explode().dropna().unique()
licensors_df = pd.DataFrame({
    'licensor_id': range(1, len(all_licensors)+1),
    'licensor_name': sorted(all_licensors)
})

# 3. Crear tabla de relación anime-género
genre_relations = []
for _, row in df_clean.iterrows():
    if pd.notna(row['genres_extracted']):
        for genre in row['genres_extracted'].split(', '):
            genre_id = genres_df[genres_df['genre_name'] == genre]['genre_id'].values[0]
            genre_relations.append({'mal_id': row['mal_id'], 'genre_id': genre_id})

anime_genres_df = pd.DataFrame(genre_relations)

# 4. Crear tabla de relación anime-tema
theme_relations = []
for _, row in df_clean.iterrows():
    if pd.notna(row['themes_extracted']):
        for theme in row['themes_extracted'].split(', '):
            theme_id = themes_df[themes_df['theme_name'] == theme]['theme_id'].values[0]
            theme_relations.append({'mal_id': row['mal_id'], 'theme_id': theme_id})

anime_themes_df = pd.DataFrame(theme_relations)

# 5. Crear tabla de relación anime-licensor (CORREGIDO)
licensors_relations = []
for _, row in df_clean.iterrows():
    if pd.notna(row['licensors_extracted']):
        for licensor in row['licensors_extracted'].split(', '):
            licensor_ids = licensors_df[licensors_df['licensor_name'] == licensor]['licensor_id'].values
            if len(licensor_ids) > 0:
                licensor_id = licensor_ids[0]
                licensors_relations.append({'mal_id': row['mal_id'], 'licensor_id': licensor_id})
            else:
                print(f"Advertencia: No se encontró el distribuidor '{licensor}' en licensors_df.")

anime_licensors_df = pd.DataFrame(licensors_relations)

# 6. Eliminar columnas originales (ya no necesarias)
df_clean = df_clean.drop(columns=['genres_extracted', 'themes_extracted', 'licensors_extracted'])

# 7. Guardar las tablas
genres_df.to_csv('genres.csv', index=False)
themes_df.to_csv('themes.csv', index=False)
licensors_df.to_csv('licensors.csv', index=False)
anime_genres_df.to_csv('anime_genres.csv', index=False)
anime_themes_df.to_csv('anime_themes.csv', index=False)
anime_licensors_df.to_csv('anime_licensors.csv', index=False)

print("✅ Estructura relacional creada:")
print(f"- genres.csv: {len(genres_df)} géneros únicos")
print(f"- themes.csv: {len(themes_df)} temas únicos")
print(f"- licensors.csv: {len(licensors_df)} distribuidoras únicos")
print(f"- anime_genres.csv: {len(anime_genres_df)} relaciones género-anime")
print(f"- anime_themes.csv: {len(anime_themes_df)} relaciones tema-anime")
print(f"- anime_licensors.csv: {len(anime_licensors_df)} relaciones distribuidora-anime")

# ================= VERIFICACIÓN FINAL =================
# Guardar el archivo principal
df_clean.to_csv('animes_clean.csv', index=False)
print("\n💾 Archivos guardados:")
print("- animes_clean.csv (datos principales)")
print("- genres.csv (tabla de géneros únicos)")
print("- anime_genres_relation.csv (relaciones anime-género)")
# ================= VERIFICACIÓN =================
print("\n✅ Dataset final:")
print(f"• Dimensiones: {df_clean.shape[0]} filas x {df_clean.shape[1]} columnas")
print(f"• Memoria usada: {df_clean.memory_usage(deep=True).sum()/1024**2:.1f} MB")
print("\n📊 Columnas principales:")
print(df_clean[first_cols].head())



🔨 Procesando géneros y temas...
✅ Estructura relacional creada:
- genres.csv: 22 géneros únicos
- themes.csv: 53 temas únicos
- licensors.csv: 92 distribuidoras únicos
- anime_genres.csv: 46293 relaciones género-anime
- anime_themes.csv: 34289 relaciones tema-anime
- anime_licensors.csv: 28404 relaciones distribuidora-anime

💾 Archivos guardados:
- animes_clean.csv (datos principales)
- genres.csv (tabla de géneros únicos)
- anime_genres_relation.csv (relaciones anime-género)

✅ Dataset final:
• Dimensiones: 27529 filas x 32 columnas
• Memoria usada: 44.1 MB

📊 Columnas principales:
   mal_id                            title   type  year  season   time_slot  \
0       1                     Cowboy Bebop     TV  1998  Spring  late_night   
1       5  Cowboy Bebop: Tengoku no Tobira  Movie  2001    Fall     unknown   
2       6                           Trigun     TV  1998  Spring  late_night   
3       7               Witch Hunter Robin     TV  2002  Summer  late_night   
4       8     

In [34]:
import pandas as pd
from datetime import datetime

# ================= CONFIGURACIÓN INICIAL =================
TIME_SLOTS = {
    1: ('morning', (5, 12)),
    2: ('midday', (12, 14)),
    3: ('afternoon', (14, 19)),
    4: ('prime', (19, 23)),
    5: ('night', (23, 24)),
    6: ('late_night', (0, 5)),
    0: ('unknown', (None, None))
}

# ================= FUNCIONES AUXILIARES =================
def get_season_from_date(date):
    try:
        month = pd.to_datetime(date).month
        if 3 <= month <= 5: return 'Spring'
        elif 6 <= month <= 8: return 'Summer'
        elif 9 <= month <= 11: return 'Fall'
        else: return 'Winter'
    except:
        return None

def get_time_slot(time_str):
    if pd.isna(time_str): return 'unknown'
    try:
        hour = int(pd.to_datetime(time_str).hour)
        for slot_id, (name, (start, end)) in TIME_SLOTS.items():
            if slot_id == 0: continue
            if (start <= hour < end) or (name == 'late_night' and (hour >= 0 or hour < 5)):
                return name
        return 'unknown'
    except:
        return 'unknown'

def extract_year_from_broadcast(row):
    try:
        if pd.notna(row['aired_from']):
            return pd.to_datetime(row['aired_from']).year
        if pd.notna(row.get('broadcast.string')):
            years = [int(s) for s in str(row['broadcast.string']).split() if s.isdigit()]
            if years: return max(years)
        if pd.notna(row.get('aired.prop.from.year')):
            return int(row['aired.prop.from.year'])
    except:
        pass
    return None

# ================= PROCESAMIENTO PRINCIPAL =================
# 1. Crear copia segura
df_clean = df.copy()

# 2. Calcular columnas nuevas
df_clean['season'] = df_clean['aired_from'].apply(get_season_from_date)
df_clean['time_slot'] = df_clean['broadcast.time'].apply(get_time_slot)
df_clean['year_computed'] = df_clean.apply(extract_year_from_broadcast, axis=1)

# 3. Consolidar columnas
df_clean['season'] = df_clean['season'].astype('category')
df_clean['time_slot'] = pd.Categorical(
    df_clean['time_slot'],
    categories=['morning', 'midday', 'afternoon', 'prime', 'night', 'late_night', 'unknown'],
    ordered=True
)
df_clean['year'] = df_clean['year'].fillna(df_clean['year_computed']).astype('Int32')

# 4. Eliminar columnas temporales/intermedias
cols_to_drop = [
    'season_computed', 'year_computed', 
    'aired.from', 'aired.to', 'aired.prop.from.day', 'aired.prop.from.month',
    'aired.prop.from.year', 'aired.prop.to.day', 'aired.prop.to.month',
    'aired.prop.to.year', 'aired.string', 'broadcast.day', 'broadcast.time',
    'broadcast.timezone', 'broadcast.string',
    # Columnas de imágenes
    'images.jpg.image_url', 'images.jpg.small_image_url', 'images.jpg.large_image_url',
    'images.webp.image_url', 'images.webp.small_image_url', 'images.webp.large_image_url',
    
    # Columnas de títulos redundantes (conservando solo title y titles_extracted)
    'title_english', 'title_japanese', 'titles', 'title_english', 'title_japanese', 'title_synonym',
    # Columnas de trailer
    'trailer.youtube_id', 'trailer.url', 'trailer.embed_url',
    'trailer.images.image_url', 'trailer.images.small_image_url',
    'trailer.images.medium_image_url', 'trailer.images.large_image_url',
    'trailer.images.maximum_image_url',
    
    # Columnas anidadas originales (usaremos las versiones _extracted)
    'genres', 'themes', 'explicit_genres', 'demographics',
    'producers', 'studios', 'licensors'
]
df_clean = df_clean.drop(columns=[col for col in cols_to_drop if col in df_clean.columns])

# 5. Optimización de tipos
dtype_map = {
    'mal_id': 'uint32',
    'episodes': 'uint16',
    'score': 'float32',
    'rank': 'uint32',
    'popularity': 'uint32',
    'members': 'uint32',
    'favorites': 'uint32',
    'decade': 'uint16'
}
df_clean = df_clean.astype(dtype_map, errors='ignore')

# 6. Reordenar columnas
first_cols = [
    'mal_id', 'title', 'type', 'year',
    'season', 'time_slot', 'decade', 'score', 'rank', 'popularity', 'members', 'favorites'
]
other_cols = [col for col in df_clean.columns if col not in first_cols]
df_clean = df_clean[first_cols + other_cols]


# ================= NORMALIZACIÓN DE GÉNEROS =================
print("\n🔨 Procesando géneros y temas...")

# 1. Extraer TODOS los géneros únicos
all_genres = df_clean['genres_extracted'].str.split(', ').explode().dropna().unique()
genres_df = pd.DataFrame({
    'genre_id': range(1, len(all_genres)+1),
    'genre_name': sorted(all_genres)
})

# 2. Extraer TODOS los temas únicos
all_themes = df_clean['themes_extracted'].str.split(', ').explode().dropna().unique()
themes_df = pd.DataFrame({
    'theme_id': range(1, len(all_themes)+1),
    'theme_name': sorted(all_themes)
})

all_licensors = df_clean['licensors_extracted'].str.split(', ').explode().dropna().unique()
licensors_df = pd.DataFrame({
    'licensor_id': range(1, len(all_licensors)+1),
    'licensor_name': sorted(all_licensors)
})

# 3. Crear tabla de relación anime-género
genre_relations = []
for _, row in df_clean.iterrows():
    if pd.notna(row['genres_extracted']):
        for genre in row['genres_extracted'].split(', '):
            genre_id = genres_df[genres_df['genre_name'] == genre]['genre_id'].values[0]
            genre_relations.append({'mal_id': row['mal_id'], 'genre_id': genre_id})

anime_genres_df = pd.DataFrame(genre_relations)

# 4. Crear tabla de relación anime-tema
theme_relations = []
for _, row in df_clean.iterrows():
    if pd.notna(row['themes_extracted']):
        for theme in row['themes_extracted'].split(', '):
            theme_id = themes_df[themes_df['theme_name'] == theme]['theme_id'].values[0]
            theme_relations.append({'mal_id': row['mal_id'], 'theme_id': theme_id})

anime_themes_df = pd.DataFrame(theme_relations)

# 5. Crear tabla de relación anime-licensor (CORREGIDO Y MEJORADO)
licensors_relations = []
for _, row in df_clean.iterrows():
    if pd.notna(row['licensors_extracted']):
        for licensor in row['licensors_extracted'].split(', '):
            licensor_ids = licensors_df[licensors_df['licensor_name'] == licensor]['licensor_id'].values
            if len(licensor_ids) > 0:
                licensor_id = licensor_ids[0]
                licensors_relations.append({'mal_id': row['mal_id'], 'licensor_id': licensor_id})
            else:
                print(f"Advertencia: No se encontró el distribuidor '{licensor}' en licensors_df para mal_id: {row['mal_id']}. Posibles errores de normalización.")
                # Opcional: Puedes decidir asignar un valor predeterminado o omitir la relación.
                # Ejemplo: licensors_relations.append({'mal_id': row['mal_id'], 'licensor_id': None})

anime_licensors_df = pd.DataFrame(licensors_relations)

# 6. Eliminar columnas originales (ya no necesarias)
df_clean = df_clean.drop(columns=['genres_extracted', 'themes_extracted', 'licensors_extracted'])

# 7. Guardar las tablas
genres_df.to_csv('genres.csv', index=False)
themes_df.to_csv('themes.csv', index=False)
licensors_df.to_csv('licensors.csv', index=False)
anime_genres_df.to_csv('anime_genres.csv', index=False)
anime_themes_df.to_csv('anime_themes.csv', index=False)
anime_licensors_df.to_csv('anime_licensors.csv', index=False)

print("✅ Estructura relacional creada:")
print(f"- genres.csv: {len(genres_df)} géneros únicos")
print(f"- themes.csv: {len(themes_df)} temas únicos")
print(f"- licensors.csv: {len(licensors_df)} distribuidoras únicos")
print(f"- anime_genres.csv: {len(anime_genres_df)} relaciones género-anime")
print(f"- anime_themes.csv: {len(anime_themes_df)} relaciones tema-anime")
print(f"- anime_licensors.csv: {len(anime_licensors_df)} relaciones distribuidora-anime")

# ================= VERIFICACIÓN FINAL =================
# Guardar el archivo principal
df_clean.to_csv('animes_clean.csv', index=False)
print("\n💾 Archivos guardados:")
print("- animes_clean.csv (datos principales)")
print("- genres.csv (tabla de géneros únicos)")
print("- anime_genres_relation.csv (relaciones anime-género)")
# ================= VERIFICACIÓN =================
print("\n✅ Dataset final:")
print(f"• Dimensiones: {df_clean.shape[0]} filas x {df_clean.shape[1]} columnas")
print(f"• Memoria usada: {df_clean.memory_usage(deep=True).sum()/1024**2:.1f} MB")
print("\n📊 Columnas principales:")
print(df_clean[first_cols].head())


🔨 Procesando géneros y temas...
✅ Estructura relacional creada:
- genres.csv: 22 géneros únicos
- themes.csv: 53 temas únicos
- licensors.csv: 92 distribuidoras únicos
- anime_genres.csv: 46293 relaciones género-anime
- anime_themes.csv: 34289 relaciones tema-anime
- anime_licensors.csv: 28404 relaciones distribuidora-anime

💾 Archivos guardados:
- animes_clean.csv (datos principales)
- genres.csv (tabla de géneros únicos)
- anime_genres_relation.csv (relaciones anime-género)

✅ Dataset final:
• Dimensiones: 27529 filas x 32 columnas
• Memoria usada: 44.1 MB

📊 Columnas principales:
   mal_id                            title   type  year  season   time_slot  \
0       1                     Cowboy Bebop     TV  1998  Spring  late_night   
1       5  Cowboy Bebop: Tengoku no Tobira  Movie  2001    Fall     unknown   
2       6                           Trigun     TV  1998  Spring  late_night   
3       7               Witch Hunter Robin     TV  2002  Summer  late_night   
4       8     

In [35]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27529 entries, 0 to 27529
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   mal_id                  27529 non-null  uint32             
 1   title                   27529 non-null  object             
 2   type                    27529 non-null  object             
 3   year                    26685 non-null  Int32              
 4   season                  27529 non-null  category           
 5   time_slot               27529 non-null  category           
 6   decade                  26685 non-null  float64            
 7   score                   27446 non-null  float32            
 8   rank                    21127 non-null  float64            
 9   popularity              27529 non-null  uint32             
 10  members                 27529 non-null  uint32             
 11  favorites               27529 non-null  uint32

In [36]:
# 1. Selección final de columnas para Tableau
tableau_cols = [
    'mal_id', 'title', 'type', 'year', 'season', 'time_slot', 'decade',
    'score', 'rank', 'popularity', 'members', 'favorites',
    'episodes', 'status', 'duration', 'rating',
    'genres_extracted', 'themes_extracted', 'studios_extracted','demographics_extracted', 'licensors_extracted',
    'synopsis', 'aired_from', 'aired_to', 
    ]

# 2. Filtrar solo las columnas que existen
existing_cols = [col for col in tableau_cols if col in df_clean.columns]
df_tableau = df_clean[existing_cols]

# 3. Limpieza final para Tableau
df_tableau = df_tableau.assign(
    # Asegurar fechas en formato Tableau
    aired_from = pd.to_datetime(df_tableau['aired_from']).dt.strftime('%Y-%m-%d'),
    aired_to = pd.to_datetime(df_tableau['aired_to']).dt.strftime('%Y-%m-%d'),
    
    # Limpiar sinopsis (eliminar saltos de línea)
    synopsis = df_tableau['synopsis'].str.replace('\n', ' ').str.replace('\s+', ' ', regex=True)
)

# 4. Guardar en CSV optimizado para Tableau
output_path = 'myanimelist_tableau_ready.csv'
df_tableau.to_csv(
    output_path,
    index=False,
    encoding='utf-8',
    date_format='%Y-%m-%d'  # Formato estándar que Tableau reconoce automáticamente
)

# 5. Verificación final
print(f"\n✅ Archivo guardado en: {output_path}")
print(f"Dimensiones finales: {df_tableau.shape[0]} filas x {df_tableau.shape[1]} columnas")
print("\nVista previa de las primeras filas:")
print(df_tableau.head(3))

# Opcional: Guardar también en Excel (mejor manejo de tipos de datos)
if 'openpyxl' in sys.modules:
    excel_path = 'myanimelist_tableau_ready.xlsx'
    df_tableau.to_excel(excel_path, index=False)
    print(f"\nVersión Excel guardada en: {excel_path}")


✅ Archivo guardado en: myanimelist_tableau_ready.csv
Dimensiones finales: 27529 filas x 21 columnas

Vista previa de las primeras filas:
   mal_id                            title   type  year  season   time_slot  \
0       1                     Cowboy Bebop     TV  1998  Spring  late_night   
1       5  Cowboy Bebop: Tengoku no Tobira  Movie  2001    Fall     unknown   
2       6                           Trigun     TV  1998  Spring  late_night   

   decade  score   rank  popularity  members  favorites  episodes  \
0  1990.0   8.75   47.0          42  1951449      85799      26.0   
1  2000.0   8.38  214.0         637   393928       1686       1.0   
2  1990.0   8.22  367.0         260   795496      16696      26.0   

            status       duration                          rating  \
0  Finished Airing  24 min per ep  R - 17+ (violence & profanity)   
1  Finished Airing    1 hr 55 min  R - 17+ (violence & profanity)   
2  Finished Airing  24 min per ep       PG-13 - Teens 13 or o